In [1]:
import numpy as np
import pandas as pd
import pma_python
from pma_python import core
import tqdm
from tqdm import tqdm
import numpy as np
from PIL import Image, ImageDraw
import shapely
from shapely import Polygon
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)
import os
import glob
import cv2
import matplotlib.pyplot as plt

In [2]:
sessionID = core.connect('hosturl','username','password')
sessionID

'pxtwTqDIO2UqNMvJAO3-9g2'

In [3]:
def find_level_x(pma_slide,sessionID,mgfx):
    level = None
    closest_level_distance = float('inf')

    for i, magnification_level in enumerate(core.get_zoomlevels_list(pma_slide, sessionID=sessionID)):
        if core.get_magnification(pma_slide, i, True, sessionID) == mgfx:
            level = i
            break
    else:
        distance = abs(core.get_magnification(pma_slide, i, True, sessionID) - mgfx)
        if distance < closest_level_distance:
            closest_level_distance = distance
            level = i

    return level

def get_scale(pma_slide,sessionID,mgfx):
    level = find_level_x(pma_slide, sessionID,mgfx) 
    max_level_dim = core.get_pixel_dimensions(pma_slide,core.get_max_zoomlevel(pma_slide,sessionID),sessionID)[0]
    zoom_level_dim = core.get_pixel_dimensions(pma_slide,level,sessionID)[0] 
    dwnsmple = zoom_level_dim / max_level_dim 
    scale = dwnsmple*(mgfx/core.get_magnification(pma_slide,level,True,sessionID))
    return scale

def get_annotations(pma_slide,sessionID,scale):
    ann = core.get_annotations(pma_slide,sessionID)
    annotations = []
    for annotation in ann:
        if "Node" in annotation['Classification']:
            annotations.append(annotation['Geometry'])
    numerical_annotations = []
    for annotation in annotations:
        coordinates_str = annotation[len('POLYGON(('):-len('))')]
        coordinate_pairs = coordinates_str.split(',')
        coordinates = [tuple(map(int, pair.strip().split())) for pair in coordinate_pairs]
        numerical_annotations.append(coordinates)
    for i in range(len(numerical_annotations)):
        annotation = numerical_annotations[i]
        scaled_annotation = [(round(x*scale), round(y*scale)) for x, y in annotation]
        numerical_annotations[i] = scaled_annotation
        numerical_annotations
    all_polygons = []
    for i in (numerical_annotations):
        lymph_node_polygon = np.array(i, dtype=np.int32)
        lymph_node_polygon = lymph_node_polygon.reshape((-1, 1, 2))
        all_polygons.append(lymph_node_polygon)
    return all_polygons


def binary_mask(pma_slide,sessionID,mgfx):
    level = find_level_x(pma_slide,sessionID,mgfx)
    scale = get_scale(pma_slide,sessionID,mgfx)
    w = core.get_slide_info(pma_slide)['Width']
    w = round(w * scale)
    h = core.get_slide_info(pma_slide)['Height']
    h = round(h * scale)
    image_shape = (h, w) #as per numpy pass image size as h,w always to get masks in right way
    
    all_polygons =  get_annotations(pma_slide,sessionID,scale)
    # Create a binary mask initialized with zeros
    binary_mask = np.zeros(image_shape[:2], dtype=np.uint8)
    
    # Fill the binary mask with the polygons
    binary_mask_1 = cv2.fillPoly(binary_mask, all_polygons, 1) 


    
    # Convert the binary mask back to PIL format (RGB)
    #binary_mask_img = Image.fromarray((binary_mask_1 * 255).astype(np.uint8))
    return binary_mask_1

In [4]:
def find_level_x(pma_slide,sessionID,mgfx):
    level = None
    closest_level_distance = float('inf')

    for i, magnification_level in enumerate(core.get_zoomlevels_list(pma_slide, sessionID=sessionID)):
        if core.get_magnification(pma_slide, i, True, sessionID) == mgfx:
            level = i
            break
    else:
        distance = abs(core.get_magnification(pma_slide, i, True, sessionID) - mgfx)
        if distance < closest_level_distance:
            closest_level_distance = distance
            level = i

    return level

def get_scale(pma_slide,sessionID,mgfx):
    level = find_level_x(pma_slide, sessionID,mgfx) 
    max_level_dim = core.get_pixel_dimensions(pma_slide,core.get_max_zoomlevel(pma_slide,sessionID),sessionID)[0]
    zoom_level_dim = core.get_pixel_dimensions(pma_slide,level,sessionID)[0] 
    dwnsmple = zoom_level_dim / max_level_dim 
    scale = dwnsmple*(mgfx/core.get_magnification(pma_slide,level,True,sessionID))
    return scale









def binary_mask_t(pma_slide,sessionID,mgfx):
    level = find_level_x(pma_slide,sessionID,mgfx)
    scale = get_scale(pma_slide,sessionID,mgfx)




    BILATERAL1_ARGS={"d":9,"sigmaColor":10000,"sigmaSpace":150}
    BILATERAL2_ARGS={"d":90,"sigmaColor":5000,"sigmaSpace":5000}
    BILATERAL3_ARGS={"d":90,"sigmaColor":10000,"sigmaSpace":10000}
    BILATERAL4_ARGS={"d":90,"sigmaColor":10000,"sigmaSpace":100}
    THRESH1_ARGS={"thresh":0,"maxval":255,"type":cv2.THRESH_TRUNC+cv2.THRESH_OTSU}
    THRESH2_ARGS={"thresh":0,"maxval":255,"type":cv2.THRESH_OTSU}
    
    w = core.get_slide_info(pma_slide)['Width']
    h = core.get_slide_info(pma_slide)['Height']
    img=core.get_region(pma_slide,x=0,y=0,width=w,height=h,scale=scale,sessionID=sessionID)
    
    img=np.array(img)
    image=img.copy()
    shape=img.shape
    img_hsv=cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
    lower_red=np.array([120,0,0])
    upper_red=np.array([180,255,255])
    mask=cv2.inRange(img_hsv,lower_red,upper_red)
    img_hsv=cv2.cvtColor(img_hsv,cv2.COLOR_HSV2RGB)
    m=cv2.bitwise_and(img,img,mask=mask)
    im_fill=np.where(m==0,233,m)
    mask=np.zeros(shape)
    gray=cv2.cvtColor(im_fill,cv2.COLOR_BGR2GRAY)
    blur1=cv2.bilateralFilter(np.bitwise_not(gray),**BILATERAL1_ARGS)
    blur2=cv2.bilateralFilter(np.bitwise_not(blur1),**BILATERAL2_ARGS)
    blur3=cv2.bilateralFilter(np.bitwise_not(blur2),**BILATERAL3_ARGS)
    blur4=cv2.bilateralFilter(np.bitwise_not(blur3),**BILATERAL4_ARGS)
    blur_final=255-blur4
    _,thresh=cv2.threshold(blur_final,**THRESH1_ARGS)
    _,thresh=cv2.threshold(thresh,**THRESH2_ARGS)
    contours,_=cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    contours=list(filter(lambda x: cv2.contourArea(x) > 9000, contours))
    new=cv2.drawContours(image,contours,-1,(0,0,255),3)
    #cv2.imwrite('new34.png',new)
   # print(len(contours))
    
    plygons =[]
    for contour in contours:
        contour = np.squeeze((1 / 32) * contour).astype(int)
        plygon = Polygon(contour)
        plygon=str(plygon)
        plygons.append(plygon)
        numerical_annotations = []
        for annotation in plygons:
            coordinates_str = annotation[len('POLYGON((('):-len('))')]
            coordinate_pairs = coordinates_str.split(',')
            coordinates = [tuple(map(int, pair.strip().split())) for pair in coordinate_pairs]
            numerical_annotations.append(coordinates)
        for i in range(len(numerical_annotations)):
            annotation = numerical_annotations[i]
            scaled_annotation = [(round(x*1/scale), round(y*1/scale)) for x, y in annotation]
            numerical_annotations[i] = scaled_annotation
            numerical_annotations
        all_plygons = []
        for i in (numerical_annotations):
            lymph_node_polygon = np.array(i, dtype=np.int32)
            lymph_node_polygon = lymph_node_polygon.reshape((-1, 1, 2))
            all_plygons.append(lymph_node_polygon)
            y = all_plygons
     
    h1 = round(h*scale)
    w1=round(w*scale)
    image_shape = (h1, w1) 
    binary_mask = np.zeros(image_shape, dtype=np.uint8)
    binary_mask_2=cv2.fillPoly(binary_mask,y, 1) 
    #binary_mask_img = cv2.imwrite('test34.png',binary_mask*255)
    return binary_mask_2 

In [5]:
cd '/drive4/Anshuman/testing_project/nodel_segmentation/'

/drive4/Anshuman/testing_project/nodel_segmentation


In [6]:
df = pd.read_csv('node_file_name.csv')

In [7]:
sessionID= core.connect('http://10.100.76.62/core/', 'tmccomputpath', 'Z5XPLQ2I')  # @office

print(sessionID)

# Loop through each slide
for i in tqdm(df["File_name"]):
    mgfx = 2
    pma_slide = i
    try:
        result1 = binary_mask(pma_slide,sessionID,mgfx)
        result2 = binary_mask_t(pma_slide,sessionID,mgfx)
        #result1 = np.resize(result1, result2.shape)
        result3 = result1*result2
        binary_mask_img = Image.fromarray((result3 * 255).astype(np.uint8))
        output_dir = 'new_masks/'  # Replace with your desired output directory path
        os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
        file_name = core.get_slide_file_name(pma_slide)
        slide_name = os.path.splitext(file_name)[0]
        output_path = f'{output_dir}/{slide_name}.png'  # Replace with your desired filename
        binary_mask_img.save(output_path)
        print (f'Mask generated for {slide_name}')
        
    except Exception as e:
        print(f"Failed to generate mask for {pma_slide}. Error: {str(e)}")
        break

pxtwTqDIO2UqNMvJAO3-9g2


  0%|            | 1/203 [00:46<2:35:57, 46.32s/it]

Mask generated for CAIB-T00000034OC01R01R0708HE


  1%|            | 2/203 [01:08<1:47:53, 32.21s/it]

Mask generated for CAIB-T00000034OC01R01R0808HE


...

Mask generated for CAIB-T00001466OP02R01R0103HE


100%|█████████▉| 202/203 [1:18:44<00:23, 23.21s/it]

Mask generated for CAIB-T00001466OP02R01R0205HE


100%|██████████| 203/203 [1:19:15<00:00, 23.43s/it]

Mask generated for CAIB-T00001466OP02R01R0309HE
